# Hypercube inducibility and statistics problems

This notebook contains calculations for inducibility and statistics problems and for hypercubes. To run these calculations, use the modified version of sage from
https://github.com/bodnalev/sage

Below is a description of the main calculation sections.

1. The inducibility Section 1 contains the description of the structures and corresponding inducibility upper bounds.

2. The statistics Section 2 contains a function to calculate statistics as linear combination of flags, and the exact upper bounds, with special discussion for $\lambda(2, 1)$.

3. The verification Section 3 contains a single cell verifying the certificates. It can be run independently from the rest of the notebook.

4. The inspect Section 4 contains code that exports pickled certificates to human readable text files. Additionally there is an interactive code to inspect the certificate data.

## Section 1 - Inducibility calculations

First cell contains representations of $W_7, W_8, W_9, W_{10}, W_{12}$ as flags in the `HypercubeVertex` theory.
The second cell gives an upper bound for the maximum induced density of the displayed $W$ structures

In [1]:
HV = Theory("HypercubeVertex")
HV.reset()
all8s = HV.generate(8)
orinds = [4, 7, 11, 6, 9]
winds = [7, 8, 9, 10, 12]

# Get a nice representation for flags
def prep(flag):
    return str(log(flag.size(), 2)) + "-subcube with points: {" + " ".join([str(xx[0]) for xx in flag.blocks("points")]) + "}"
# Confirm the structures are the same as claimed
print("The structures are:")
for ii in range(len(winds)):
    print("W{} is {}".format(winds[ii], prep(all8s[orinds[ii]])))

The structures are:
W7 is 3-subcube with points: {0 7}
W8 is 3-subcube with points: {0 3 5}
W9 is 3-subcube with points: {0 1 2 7}
W10 is 3-subcube with points: {0 1 6}
W12 is 3-subcube with points: {0 1 2 4}


In [2]:
# Calculate the bound for each
HV.reset()
target_sizes = [8, 16, 8, 16, 8]
for ii in range(len(winds)):
    lam_w_ii = HV.optimize(all8s[orinds[ii]], target_sizes[ii], exact=True, 
                           file="certificates/w{}_cert".format(winds[ii]), 
                           printlevel=0
                          )
    print("For W{} the found upper bound is {}".format(winds[ii], lam_w_ii))

For W7 the found upper bound is 1/3
For W8 the found upper bound is 2/3
For W9 the found upper bound is 4/9
For W10 the found upper bound is 5/12
For W12 the found upper bound is 1/2


## Section 2 - Statistics calculations

First a `targ_ds` function is defined, it creates a quantum flag representing the sum of hypercubes with dimension $d$ and $s$ points. Then the induced exact upper-bound for various $d, s$ pairs are calculated.

The following Subsection 2.1 deals with $\lambda(2,1)$, by arguing that the $24/35$ flag upper-bound can not be tight. First calculate the configurations with zero slack $H$ coming from the $24/35$ bound. By determining that $H_5$ has no valid extension among the zero slack patterns, a final maximization of $\lambda(2, 1)$ without $H_5$ yields a strictly smaller bound, showing that $24/35$ is not attainable.

Finally, Subsection 2.2 displays the floating-point data presented in the Table 1, proving unrounded upper-bounds for the remaining statistics cases.

In [3]:
# Define the structures for the statistics
HV.reset()
def targ_ds(d, s):
    return sum([xx for xx in HV.generate(2**d) if len(xx.blocks("points"))==s])

for d, s in [(3, 2), (4, 2), (4, 4)]:
    lam_ds = HV.optimize(targ_ds(d, s), 16, exact=True, 
                         file="certificates/d{}s{}_cert".format(d, s), 
                         printlevel=0
                        )
    print("For d={} and s={} the found upper bound is {}".format(d, s, lam_ds))

For d=3 and s=2 the found upper bound is 8/9
For d=4 and s=2 the found upper bound is 264/343
For d=4 and s=4 the found upper bound is 26/27


### Subsection 2.1 - Proof that lambda(2, 1) is not 24/35

In [5]:
HV.reset()
lam_21 = HV.optimize(targ_ds(2, 1), 16, exact=True, 
                     file="certificates/d2s1_cert",
                     printlevel=0
                    )
print("For d=2 and s=1 the found upper bound is {}".format(lam_21))
import pickle
with open("certificates/d2s1_cert.pickle", "rb") as file:
    certificate = pickle.load(file)
# Get the flags with zero slack in the certificate
H = [
    HV(certificate["base flags"][ii][0], **dict(certificate["base flags"][ii][2])) for 
    ii,vv in enumerate(certificate["slack vector"]) if QQ(vv)==0
]
print("The configurations with zero slack at the upper bound above:")
print("\n".join(["H_" + str(ii+1) + " is the " + prep(xx) for ii,xx in enumerate(H)]))

For d=2 and s=1 the found upper bound is 24/35
The configurations with zero slack at the upper bound above:
H_1 is the 4-subcube with points: {}
H_2 is the 4-subcube with points: {0 3 5 6}
H_3 is the 4-subcube with points: {0 3 13 14}
H_4 is the 4-subcube with points: {0 3 5 9 14}
H_5 is the 4-subcube with points: {0 3 5 10 12}
H_6 is the 4-subcube with points: {0 3 5 6 9 10 12 15}


In [19]:
from functools import lru_cache

@lru_cache
def generate_hypercube_edges(d):
    edges = []
    for vertex in range(2**d):
        for i in range(d):
            if not (vertex >> i) & 1:
                neighbor = vertex | (1 << i)
                edges.append((vertex, neighbor))
    return tuple(edges)

def remove_bit(x, i):
    return ((x >> (i + 1)) << i) | (x & ((1 << i) - 1))

def restrict_to_face(flag, i, b):
    face_points = []
    d = log(flag.size(), 2)
    for xx in flag.blocks("points"):
        x = xx[0]
        if ((x >> i) & 1) == b:
            y = remove_bit(x, i)
            face_points.append([y])
    return HV(flag.size()/2, edges=generate_hypercube_edges(d-1), points=face_points)

def stack_flags(base, extension):
    d = log(base.size(), 2)
    points = []
    points.extend(base.blocks("points"))
    points.extend([[(1 << d) + v[0]] for v in extension.blocks("points")])
    return HV(2**(d+1), edges=generate_hypercube_edges(d+1), points=points)

def all_automorphisms(d):
    perms = list(itertools.permutations(range(d)))
    refls = list(itertools.product([0, 1], repeat=d))
    automorphisms = []
    for perm in perms:
        for refl in refls:
            automorphisms.append((perm, refl))
    return automorphisms

def apply_automorphism(flag, automorphism):
    perm, refl = automorphism
    d = log(flag.size(), 2)
    new_points = []
    for vv in flag.blocks("points"):
        v = vv[0]
        new_v = 0
        for i in range(d):
            bit = (v >> i) & 1
            new_bit = bit ^^ refl[i]
            new_v += new_bit * (1 << perm[i])
        new_points.append([new_v])
    return HV(2**d, edges=generate_hypercube_edges(d), points=new_points)

def distinct_automorphism_variants(flag):
    d = log(flag.size(), 2)
    autos = all_automorphisms(d)
    variants = {}
    for auto in autos:
        variant = apply_automorphism(flag, auto)
        key = tuple([xx[0] for xx in variant.blocks("points")])
        if key not in variants:
            variants[key] = variant
    return list(variants.values())

def generate_extensions(flag, H):
    d = log(flag.size(), 2)
    valid_configs = []
    # All non-equal permutations of the flag
    flag_variants = distinct_automorphism_variants(flag)
    # Try to stack every element from H
    for base in H:
        # With every permutation of the flag
        for flag_variant in flag_variants:
            candidate = stack_flags(base, flag_variant)
            valid = True
            for i in range(d+1):
                for b in (0, 1):
                    # Check if the faces are still in H
                    face = restrict_to_face(candidate, i, b)
                    if face not in H:
                        valid = False
                        break
                if not valid:
                    break
            if valid:
                print("Found a valid extension: ", candidate)
                valid_configs.append(candidate)
    return valid_configs

# Print the possibe extensions of H_5 that only contains subcubes from H
print(generate_extensions(H[4], H))

[]


In [20]:
# Optimize lambda(2, 1), but now assuming H_5 has density 0
HV.exclude(H[4])
lam_21_noh5 = HV.optimize(targ_ds(2, 1), 16, exact=True, denom=2**20,
                     file="certificates/d2s1_noh5_cert",
                     printlevel=0
                    )
print("Without H_5 the optimum is {} ~= {}. (Note {} ~= {})".format(lam_21_noh5, lam_21_noh5.n(), 24/35, (24/35).n()))

Without H_5 the optimum is 528013/786432 ~= 0.671403249104818. (Note 24/35 ~= 0.685714285714286)


### Subsection 2.2 - Remaining small statistics cases
An unrounded upper-bound is provided for each remaining $d \leq 4$ statistics case.

In [21]:
HV.reset()
for d, s in [(3, 1), (3, 3), (4, 1), (4, 3), (4, 5), (4, 6), (4, 7)]:
    lam_ds = HV.optimize(targ_ds(d, s), 16, exact=False, printlevel=0)
    print("For d={} and s={} the found numerical upper bound is {}".format(d, s, lam_ds))

For d=3 and s=1 the found numerical upper bound is 0.6100426093596065
For d=3 and s=3 the found numerical upper bound is 0.6842325624118085
For d=4 and s=1 the found numerical upper bound is 0.6025329211355644
For d=4 and s=3 the found numerical upper bound is 0.681263540657437
For d=4 and s=5 the found numerical upper bound is 0.7096421914321209
For d=4 and s=6 the found numerical upper bound is 0.8540599904128074
For d=4 and s=7 the found numerical upper bound is 0.7270395543425598


## Section 3 - Verification of the certificates

If one only wants to verify that any of the exact certificates are indeed correct, it is enough
to run this cell. For each step above, it loads the generated certificates and verifies that the matrices are indeed positive semidefinite
and that the bound they prove is exactly as claimed. Note however that the majority of the time is spent at calculating the problem data, once that is complete all the cells run quickly and it is not much slower to re-calculate the certificates instead of verifying them.

In [22]:
HV = Theory("HypercubeVertex")
HV.reset()
all8s = HV.generate(8)
orinds = [4, 7, 11, 6, 9]
winds = [7, 8, 9, 10, 12]
target_sizes = [8, 16, 8, 16, 8]
for ii in range(len(winds)):
    print("Verifying for W{}".format(winds[ii]))
    HV.verify("certificates/w{}_cert".format(winds[ii]), 
              all8s[orinds[ii]], target_sizes[ii], printlevel=1
             )
def targ_ds(d, s):
    return sum([xx for xx in HV.generate(2**d) if len(xx.blocks("points"))==s])
for d, s in [(3, 2), (4, 2), (4, 4)]:
    print("Verifying for d={}, s={}".format(d, s))
    HV.verify("certificates/d{}s{}_cert".format(d, s), 
              targ_ds(d, s), 16, printlevel=1
             )
H5 = HV.generate(16)[57]
print("H_5 is ", H5)
HV.exclude(H5)
HV.verify("certificates/d2s1_noh5_cert", 
          targ_ds(2, 1), 16, printlevel=1
         )

Verifying for W7
Checking X matrices


5it [00:00, 4312.47it/s]


Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


5it [00:00, 2345.28it/s]


Done calculating linear constraints
Calculating the bound provided by the certificate


...

13it [00:00, 290.85it/s]


Done calculating linear constraints
Calculating the bound provided by the certificate


13it [00:01,  9.57it/s]

The solution is valid, it proves the bound 528013/786432


528013/786432

## Section 4 - Inspect certificates interactively

The largest semidefinite block has dimension 16, the last part allows the interactive inspection of the certificate. An easy to read text file is also exported to the certificates folder, for each exact result.

First, an easy to read data is exported. They are simple text files, showing the linear combination of typed configurations, whose (scaled) square provides the final proof in each case.

These terms can be inspected afterwards, in an interactive way as well, by running the final cell.

In [1]:
import pickle
from sage.algebras.combinatorial_theory import _unflatten_matrix
HV = Theory("HypercubeVertex")
HV.reset()

def hprint(hflag):
    dim = int(log(hflag.size(), 2))
    points = [xx[0] for xx in hflag.blocks("points")]
    type = hflag.ftype_points()
    return f"({dim}-dim cube with {type} type and {points} marked points)"
def vecprint(vec, tdata, hide=True):
    flags = HV.generate(tdata[0], tdata[1])
    return "\n".join([f"{ww} \t {hprint(flags[ii])}" for ii,ww in enumerate(vec) if ww!=0 or (not hide)])

cert_names = ["w7", "w8", "w9", "w10", "w12", "d2s1", "d2s1_noh5", "d3s2", "d4s2", "d4s4"]
interactive_data = {}

for nind,name in enumerate(cert_names):
    cert = pickle.load(open(f"certificates/{name}_cert.pickle", "rb"))
    bound = QQ(cert["result"])
    tsize = cert["target size"]
    base_strs = HV.generate(tsize)
    base_hprint = "\n".join(map(hprint, base_strs))
    slacks_hprint = vecprint(vector(cert["slack vector"]), [tsize, None], False)
    types = HV._get_relevant_ftypes(tsize)
    slacks = cert["slack vector"]
    int_data = {
        "0 - the final bound": bound,
        "1 - the vertices of the cubes used": tsize, 
        "2 - the list of maximum sized hypercubes used": base_hprint, 
        "3 - the list of slack values": slacks_hprint
    }
    squares = {}
    with open(f"certificates/{name}_cert.txt", "w") as file:
        file.write(f"Text certificate for {name} proving the bound {bound}\n{"="*50}\n\n")
        file.write(f"Base structures\n{base_hprint}\n{"="*50}\n\n")
        file.write(f"Slack values\n{slacks_hprint}\n{"="*50}\n\n")
        file.write(f"Semi-definite blocks\n")
        for ii,tdata in enumerate(types):
            mat = _unflatten_matrix(cert["X matrices"][ii])[0]
            P, L, D = mat.block_ldlt()
            type_hprint = hprint(tdata[1])
            type_interactive = {}
            file.write(f"{"-"*50}\n\nType {type_hprint} contributes the following")
            all_squares = ""
            for jj,col in enumerate((P*L).columns()):
                text = f"\n{D[jj,jj]} scaled square of \n{vecprint(col, tdata)}\n"
                all_squares += text
                type_interactive[f"{jj} - the square with this index and it's scale"] = text
                file.write(text)
            type_interactive[f"{P.nrows()} - all the squares and their scale"] = all_squares
            squares[f"{ii} - squares coming from type {type_hprint}"] = type_interactive
    int_data["4 - the squares used in the proof"] = squares
    interactive_data[f"{nind} - certificate for {name}"] = int_data

In [2]:
def inspect(dd):
    keys = list(dd.keys())
    for kk in keys:
        print(kk)
    print("\n")
    val = input("Inspect: ")
    res = dd[keys[int(val)]]
    print("\n")
    if isinstance(res, dict):
        inspect(res)
    else:
        print(res)

inspect(interactive_data)

0 - certificate for w7
1 - certificate for w8
2 - certificate for w9
3 - certificate for w10
4 - certificate for w12
5 - certificate for d2s1
6 - certificate for d2s1_noh5
7 - certificate for d3s2
8 - certificate for d4s2
9 - certificate for d4s4




Inspect:  4




0 - the final bound
1 - the vertices of the cubes used
2 - the list of maximum sized hypercubes used
3 - the list of slack values
4 - the squares used in the proof




Inspect:  4




0 - squares coming from type (0-dim cube with (0,) type and [] marked points)
1 - squares coming from type (0-dim cube with (0,) type and [0] marked points)
2 - squares coming from type (1-dim cube with (0, 1) type and [] marked points)
3 - squares coming from type (1-dim cube with (0, 1) type and [0] marked points)
4 - squares coming from type (1-dim cube with (0, 1) type and [0, 1] marked points)




Inspect:  4




0 - the square with this index and it's scale
1 - the square with this index and it's scale
2 - the square with this index and it's scale
3 - the square with this index and it's scale
4 - all the squares and their scale




Inspect:  4





615/1024 scaled square of 
1 	 (2-dim cube with (0, 1) type and [0, 1] marked points)
-2/41 	 (2-dim cube with (0, 1) type and [0, 1, 2, 3] marked points)

77/384 scaled square of 
1 	 (2-dim cube with (0, 1) type and [0, 1, 2] marked points)
-1 	 (2-dim cube with (1, 0) type and [0, 1, 2] marked points)

0 scaled square of 
1 	 (2-dim cube with (1, 0) type and [0, 1, 2] marked points)

6787/41984 scaled square of 
1 	 (2-dim cube with (0, 1) type and [0, 1, 2, 3] marked points)

